#HW3 - SQL
 
This homework has you working with a new database of information on ticket sales for various types of events.  Your job will be to do some initial exploring and then demonstrate your ability to do all the different types of SQL queries we learned over the last two week.  You'll also need to make one function that'll make looking at the tables easier. 
 
These questions are written in the way someone would ask them to you.  In other words, I'm using 'plain english' questions vs. ones where I'm very explicit in terms of what columns and tables to use.  Your exploring of the database and functions to ease that process will come in handy here!  
 
The database has been created using a set of data from Amazon. You can read more about what each table contains here: https://docs.aws.amazon.com/redshift/latest/dg/c_sampledb.html.  

## Libraries and import functions

First bring the libraries we'll need!

In [161]:
import psycopg2 
import pandas as pd

Now bring in all our functions we used in the lessons!  

In [162]:
# Make our connection/cursor function 
AWS_host_name = "ticketsdb.chzr8692xwjt.us-east-2.rds.amazonaws.com"
AWS_dbname = "ticketsdb"
AWS_user_name = "postgres"
AWS_password = "ista322ticketsdb"

def get_conn_cur(): # define function name and arguments (there aren't any)
  # Make a connection
  conn = psycopg2.connect(
    host=AWS_host_name,
    database=AWS_dbname,
    user=AWS_user_name,
    password=AWS_password,
    port='5432')
  
  cur = conn.cursor()   # Make a cursor after

  return(conn, cur)   # Return both the connection and the cursor

# Same run_query function
def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  # here we're extracting the 0th element for each item in cur.description
  colnames = [desc[0] for desc in cur.description]

  cur.close() # close
  conn.close() # close

  return(colnames, my_data) # return column names AND data

# Column name function for checking out what's in a table
def get_column_names(table_name): # arguement of table_name
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  column_name_query =  """SELECT column_name FROM information_schema.columns
       WHERE table_name = '%s' """ %table_name

  cur.execute(column_name_query) # exectue
  my_data = cur.fetchall() # store

  cur.close() # close
  conn.close() # close

  return(my_data) # return

# Check table_names
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results

## Make a SQL head function - 5 point

Make function to get the pandas equivalent of `.head()`

This function should be called `sql_head` and take a single argument of `table_name` where you specify the table name you want the head information from.  It should return the column names along with the first five rows of the table along.  

**For full points, return a pandas dataframe with this information so it displays nicely :)**

In [163]:
# make sql_head function
def sql_head(table_name):
  # columns = get_column_names(table_name)
  limit = 5
  sql = f"select * from {table_name} limit {limit}"
  results = run_query(sql)
  columns = results[0]
  data = results[1]
  df = pd.DataFrame(columns=columns, data=data)
  return df

In [164]:
# Check that it works!
sql_head(table_name = 'sales')

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,pricepaid,commission,saletime
0,1,1,36861,21191,7872,1875,4,728.00,109.20,2008-02-18 02:36:48
1,2,4,8117,11498,4337,1983,2,76.00,11.40,2008-06-06 05:00:16
2,3,5,1616,17433,8647,1983,2,350.00,52.50,2008-06-06 08:26:17
3,4,5,1616,19715,8647,1986,1,175.00,26.25,2008-06-09 08:38:52
4,5,6,47402,14115,8240,2069,2,154.00,23.10,2008-08-31 09:17:02


## Explore and SELECT - 5 point

Let's start this homework with some basic queries to get a look at what's in the various tables.  I want you to do the following.

* Use your get_table_names() function to see what tables are in the database.
* Use your get_column_names() to get the column names of each of the tables.  **Do this all within a single cell to keep it neat**.
* Make and run a query that selects all columns from the event table.  Only return the first 5 rows.
* Use the `sql_head()` function you created to get the first five rows of the sales table.

In [165]:
# Getting table names
table_info = []
table_names = get_table_names()
table_names

[('users',),
 ('venue',),
 ('category',),
 ('date',),
 ('event',),
 ('listing',),
 ('sales',)]

In [166]:
# Getting column names
table_info = []
for tn in table_names:
  columns = get_column_names(tn)
  table_info.append([tn, columns])

for row in table_info:
  print(row)

[('users',), [('userid',), ('username',), ('firstname',), ('lastname',), ('city',), ('state',), ('email',), ('phone',), ('likesports',), ('liketheatre',), ('likeconcerts',), ('likejazz',), ('likeclassical',), ('likeopera',), ('likerock',), ('likevegas',), ('likebroadway',), ('likemusicals',)]]
[('venue',), [('venueid',), ('venuename',), ('venuecity',), ('venuestate',), ('venueseats',)]]
[('category',), [('catid',), ('catgroup',), ('catname',), ('catdesc',)]]
[('date',), [('dateid',), ('caldate',), ('day',), ('week',), ('month',), ('qtr',), ('year',), ('holiday',)]]
[('event',), [('eventid',), ('venueid',), ('catid',), ('dateid',), ('eventname',), ('starttime',)]]
[('listing',), [('listid',), ('sellerid',), ('eventid',), ('dateid',), ('numtickets',), ('priceperticket',), ('totalprice',), ('listtime',)]]
[('sales',), [('salesid',), ('listid',), ('sellerid',), ('buyerid',), ('eventid',), ('dateid',), ('qtysold',), ('pricepaid',), ('commission',), ('saletime',)]]


In [167]:
# Could also just use a list comprehension vs a bunch of print statements :)
names = get_table_names()
[get_column_names(table_name= x) for x in names]

[[('userid',),
  ('username',),
  ('firstname',),
  ('lastname',),
  ('city',),
  ('state',),
  ('email',),
  ('phone',),
  ('likesports',),
  ('liketheatre',),
  ('likeconcerts',),
  ('likejazz',),
  ('likeclassical',),
  ('likeopera',),
  ('likerock',),
  ('likevegas',),
  ('likebroadway',),
  ('likemusicals',)],
 [('venueid',),
  ('venuename',),
  ('venuecity',),
  ('venuestate',),
  ('venueseats',)],
 [('catid',), ('catgroup',), ('catname',), ('catdesc',)],
 [('dateid',),
  ('caldate',),
  ('day',),
  ('week',),
  ('month',),
  ('qtr',),
  ('year',),
  ('holiday',)],
 [('eventid',),
  ('venueid',),
  ('catid',),
  ('dateid',),
  ('eventname',),
  ('starttime',)],
 [('listid',),
  ('sellerid',),
  ('eventid',),
  ('dateid',),
  ('numtickets',),
  ('priceperticket',),
  ('totalprice',),
  ('listtime',)],
 [('salesid',),
  ('listid',),
  ('sellerid',),
  ('buyerid',),
  ('eventid',),
  ('dateid',),
  ('qtysold',),
  ('pricepaid',),
  ('commission',),
  ('saletime',)]]

In [168]:
# Query on events
sq = """select * from event limit 5"""
run_query(sq)

(['eventid', 'venueid', 'catid', 'dateid', 'eventname', 'starttime'],
 [(1, 305, 8, 1851, 'Gotterdammerung', datetime.datetime(2008, 1, 25, 14, 30)),
  (2, 306, 8, 2114, 'Boris Godunov', datetime.datetime(2008, 10, 15, 20, 0)),
  (3, 302, 8, 1935, 'Salome', datetime.datetime(2008, 4, 19, 14, 30)),
  (4,
   309,
   8,
   2090,
   'La Cenerentola (Cinderella)',
   datetime.datetime(2008, 9, 21, 14, 30)),
  (5, 302, 8, 1982, 'Il Trovatore', datetime.datetime(2008, 6, 5, 19, 0))])

In [169]:
# Use sql_head to get the head of sales
sql_head("sales")

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,pricepaid,commission,saletime
0,1,1,36861,21191,7872,1875,4,728.00,109.20,2008-02-18 02:36:48
1,2,4,8117,11498,4337,1983,2,76.00,11.40,2008-06-06 05:00:16
2,3,5,1616,17433,8647,1983,2,350.00,52.50,2008-06-06 08:26:17
3,4,5,1616,19715,8647,1986,1,175.00,26.25,2008-06-09 08:38:52
4,5,6,47402,14115,8240,2069,2,154.00,23.10,2008-08-31 09:17:02


## WHERE - 5 points

Now let's do a bit of filtering with WHERE.  Write and run queries to get the following results.  
**LIMIT all returns to first five rows.**

* Get venues with >= 10000 seats from the venues table
* Get venues in Arizona
* Get users who have a first name that starts with H
* Get **just email addresses** of users who gave a .edu email address




In [170]:
# Get big venues... so those with >= than 10000 seats
sq = """select * from venue where venueseats >= 10000"""
run_query(sq)

(['venueid', 'venuename', 'venuecity', 'venuestate', 'venueseats'],
 [(5, 'Gillette Stadium', 'Foxborough', 'MA', 68756),
  (6, 'New York Giants Stadium', 'East Rutherford', 'NJ', 80242),
  (15, 'McAfee Coliseum', 'Oakland', 'CA', 63026),
  (18, 'Madison Square Garden', 'New York City', 'NY', 20000),
  (67, 'Ralph Wilson Stadium', 'Orchard Park', 'NY', 73967),
  (68, 'Rogers Centre', 'Toronto', 'ON', 50516),
  (69, 'Dolphin Stadium', 'Miami Gardens', 'FL', 74916),
  (70, 'M&T Bank Stadium', 'Baltimore', 'MD', 70107),
  (71, 'Paul Brown Stadium', 'Cincinnati', 'OH', 65535),
  (72, 'Cleveland Browns Stadium', 'Cleveland', 'OH', 73200),
  (73, 'Heinz Field', 'Pittsburgh', 'PA', 65050),
  (74, 'Reliant Stadium', 'Houston', 'TX', 72000),
  (75, 'Lucas Oil Stadium', 'Indianapolis', 'IN', 63000),
  (76, 'Jacksonville Municipal Stadium', 'Jacksonville', 'FL', 73800),
  (77, 'LP Field', 'Nashville', 'TN', 68804),
  (78, 'INVESCO Field', 'Denver', 'CO', 76125),
  (79, 'Arrowhead Stadium', 'Kansa

In [171]:
# Get venues in AZ
sq = """select * from venue where venuestate = 'AZ'"""
run_query(sq)

(['venueid', 'venuename', 'venuecity', 'venuestate', 'venueseats'],
 [(38, 'US Airways Center', 'Phoenix', 'AZ', 0),
  (65, 'Jobing.com Arena', 'Glendale', 'AZ', 0),
  (92, 'University of Phoenix Stadium', 'Glendale', 'AZ', 0),
  (117, 'Chase Field', 'Phoenix', 'AZ', 0)])

In [172]:
# Get users who have a first name that starts with H
sq = """select * from users where firstname like 'H%'"""
run_query(sq)

(['userid',
  'username',
  'firstname',
  'lastname',
  'city',
  'state',
  'email',
  'phone',
  'likesports',
  'liketheatre',
  'likeconcerts',
  'likejazz',
  'likeclassical',
  'likeopera',
  'likerock',
  'likevegas',
  'likebroadway',
  'likemusicals'],
 [(13,
   'QTF33MCG',
   'Henry',
   'Cochran',
   'Bossier City',
   'QC',
   'Aliquam.vulputate.ullamcorper@amalesuada.org',
   '(783) 105-0989',
   None,
   True,
   None,
   None,
   None,
   None,
   True,
   True,
   True,
   None),
  (22,
   'RHT62AGI',
   'Hermione',
   'Trevino',
   'Walnut',
   'WI',
   'non.justo.Proin@ametconsectetuer.edu',
   '(245) 110-6540',
   None,
   True,
   None,
   False,
   False,
   None,
   None,
   True,
   False,
   None),
  (29,
   'HUH27PKK',
   'Helen',
   'Avery',
   'Garland',
   'PE',
   'in.faucibus.orci@ultrices.edu',
   '(385) 925-3875',
   None,
   False,
   None,
   True,
   None,
   None,
   None,
   True,
   False,
   None),
  (56,
   'MHU11LZP',
   'Howard',
   'Wiley',
 

In [173]:
# Get all .edu email addresses... just the email addresses
sq = """select email from users where email like '%.edu'"""
run_query(sq)

(['email'],
 [('Etiam.laoreet.libero@sodalesMaurisblandit.edu',),
  ('Suspendisse.tristique@nonnisiAenean.edu',),
  ('ullamcorper.nisl@Cras.edu',),
  ('vel.est@velitegestas.edu',),
  ('justo.nec.ante@quismassa.edu',),
  ('non.justo.Proin@ametconsectetuer.edu',),
  ('non@enimsitamet.edu',),
  ('cursus@pedeacurna.edu',),
  ('in.faucibus.orci@ultrices.edu',),
  ('magnis.dis.parturient@iaculisodioNam.edu',),
  ('sapien.molestie.orci@Vestibulumaccumsan.edu',),
  ('risus.Donec.egestas@Suspendissecommodo.edu',),
  ('mi.lorem@nunc.edu',),
  ('enim.gravida@placerataugue.edu',),
  ('in.lobortis.tellus@bibendumDonecfelis.edu',),
  ('quis@sitamet.edu',),
  ('ornare.facilisis.eget@sociisnatoquepenatibus.edu',),
  ('egestas.a.scelerisque@utnisi.edu',),
  ('dictum.augue.malesuada@arcueu.edu',),
  ('augue.scelerisque@pellentesque.edu',),
  ('sollicitudin.a@Nuncut.edu',),
  ('Vivamus.sit@fermentum.edu',),
  ('enim.gravida@atnisi.edu',),
  ('neque.tellus.imperdiet@Fuscealiquam.edu',),
  ('aliquam@euarcu

## GROUP BY and HAVING - 5 points
 
Time to practice some GROUP BY and HAVING operations. Please write and run queries that do the following:
 
GROUP BY application
* Find the top five venues that hosted the most events: Alias the count of events as 'events_hosted'. Also return the venue ID
* Get the number of events hosted in each month. You'll need to use `date_part()` in your select to select just the months. Alias this as 'month' and then the count of the number of events hosted as 'events_hosted'. 
* Get the top five sellers who made the most commission. Alias their total commission made as 'total_com'. Also get their average commission made and alias as 'avg_com'.  Be sure to also display the seller_id.  
 
HAVING application
* Using the same query as the last one, instead of getting the top five sellers get all sellers who have made a total commission greater than $4000.
* Using the same query as the first groupby, instead of returning the top five venues, return just the ID's of venues that have had greater than 60 events. 

In [174]:
### GROUP BY application
# Find the top five venues that hosted the most events: Alias the count of events as 'events_hosted'. Also return the venue ID
sql_head("event")
sq = """select venueid, count(venueid) as events_hosted from event group by venueid order by events_hosted desc limit 5"""
run_query(sq)

(['venueid', 'events_hosted'],
 [(220, 81), (217, 81), (203, 80), (222, 74), (216, 72)])

In [175]:
# Get the number of events hosted in each month. You'll need to use `date_part()` in your select to select just the months.
# Alias this as 'month' and then the count of the number of events hosted as 'events_hosted'
sq = """select date_part('month', starttime) as month, count(eventid) as events_hosted from event group by month order by month asc"""
run_query(sq)

(['month', 'events_hosted'],
 [(1.0, 778),
  (2.0, 711),
  (3.0, 753),
  (4.0, 725),
  (5.0, 727),
  (6.0, 709),
  (7.0, 729),
  (8.0, 737),
  (9.0, 746),
  (10.0, 735),
  (11.0, 726),
  (12.0, 722)])

In [176]:
# Get the top five sellers who made the most commission. Alias their total commission made as 'total_com'. 
# Also get their average commission made and alias as 'avg_com'. Be sure to also display the seller_id
sq = """select sellerid, sum(commission) as total_com, avg(commission) as avg_com from sales group by sellerid order by total_com desc limit 5"""
run_query(sq)

(['sellerid', 'total_com', 'avg_com'],
 [(1140, Decimal('4859.85'), Decimal('347.1321428571428571')),
  (43551, Decimal('4704.75'), Decimal('470.4750000000000000')),
  (13385, Decimal('4274.25'), Decimal('388.5681818181818182')),
  (25433, Decimal('4147.95'), Decimal('518.4937500000000000')),
  (2372, Decimal('4073.85'), Decimal('678.9750000000000000'))])

In [177]:
### HAVING application
# Using the same query as the last groupby, instead of getting the top five sellers get all sellers who have made a total commission greater than $4000
sq = """select sellerid, sum(commission) as total_com, avg(commission) as avg_com from sales group by sellerid having sum(commission) > 4000 order by total_com desc"""
run_query(sq)

(['sellerid', 'total_com', 'avg_com'],
 [(1140, Decimal('4859.85'), Decimal('347.1321428571428571')),
  (43551, Decimal('4704.75'), Decimal('470.4750000000000000')),
  (13385, Decimal('4274.25'), Decimal('388.5681818181818182')),
  (25433, Decimal('4147.95'), Decimal('518.4937500000000000')),
  (2372, Decimal('4073.85'), Decimal('678.9750000000000000'))])

In [178]:
# Using the same query as the first groupby, instead of returning the top five venues, return just the ID's of venues that have had greater than 60 events
sq = """select venueid from event group by venueid having count(venueid) > 60"""
run_query(sq)

(['venueid'],
 [(237,),
  (238,),
  (239,),
  (226,),
  (209,),
  (205,),
  (201,),
  (216,),
  (220,),
  (221,),
  (207,),
  (243,),
  (222,),
  (248,),
  (208,),
  (217,),
  (218,),
  (215,),
  (203,),
  (228,)])

## JOIN - 5 points
 
Time for some joins. You've probably noticed by now that there is at least one relational key in each table, but some have more.  For example, sales has a unique sale id, listing id, seller id, buyer id, date id.  This allows you to link each sale to relevant information in other tables.  
**LIMIT each output to 5**
 
Please write queries to do the following items:
 
* Join information of users to each sale made.  
* Join information about each venue to each event. 

In [179]:
sq = """ SELECT * FROM sales
          LIMIT 5;"""
run_query(sq)

(['salesid',
  'listid',
  'sellerid',
  'buyerid',
  'eventid',
  'dateid',
  'qtysold',
  'pricepaid',
  'commission',
  'saletime'],
 [(1,
   1,
   36861,
   21191,
   7872,
   1875,
   4,
   Decimal('728.00'),
   Decimal('109.20'),
   datetime.datetime(2008, 2, 18, 2, 36, 48)),
  (2,
   4,
   8117,
   11498,
   4337,
   1983,
   2,
   Decimal('76.00'),
   Decimal('11.40'),
   datetime.datetime(2008, 6, 6, 5, 0, 16)),
  (3,
   5,
   1616,
   17433,
   8647,
   1983,
   2,
   Decimal('350.00'),
   Decimal('52.50'),
   datetime.datetime(2008, 6, 6, 8, 26, 17)),
  (4,
   5,
   1616,
   19715,
   8647,
   1986,
   1,
   Decimal('175.00'),
   Decimal('26.25'),
   datetime.datetime(2008, 6, 9, 8, 38, 52)),
  (5,
   6,
   47402,
   14115,
   8240,
   2069,
   2,
   Decimal('154.00'),
   Decimal('23.10'),
   datetime.datetime(2008, 8, 31, 9, 17, 2))])

In [180]:
sq = """ SELECT * FROM users
          LIMIT 5;"""
run_query(sq)

(['userid',
  'username',
  'firstname',
  'lastname',
  'city',
  'state',
  'email',
  'phone',
  'likesports',
  'liketheatre',
  'likeconcerts',
  'likejazz',
  'likeclassical',
  'likeopera',
  'likerock',
  'likevegas',
  'likebroadway',
  'likemusicals'],
 [(1,
   'JSG99FHE',
   'Rafael',
   'Taylor',
   'Kent',
   'WA',
   'Etiam.laoreet.libero@sodalesMaurisblandit.edu',
   '(664) 602-4412',
   True,
   True,
   None,
   False,
   True,
   None,
   None,
   True,
   False,
   True),
  (2,
   'PGL08LJI',
   'Vladimir',
   'Humphrey',
   'Murfreesboro',
   'SK',
   'Suspendisse.tristique@nonnisiAenean.edu',
   '(783) 492-1886',
   None,
   None,
   None,
   True,
   True,
   None,
   None,
   True,
   False,
   True),
  (3,
   'IFT66TXU',
   'Lars',
   'Ratliff',
   'High Point',
   'ME',
   'amet.faucibus.ut@condimentumegetvolutpat.ca',
   '(624) 767-2465',
   True,
   False,
   None,
   False,
   None,
   False,
   True,
   None,
   None,
   True),
  (4,
   'XDZ38RDD',
   'Barr

In [181]:
get_column_names(table_name='users')

[('userid',),
 ('username',),
 ('firstname',),
 ('lastname',),
 ('city',),
 ('state',),
 ('email',),
 ('phone',),
 ('likesports',),
 ('liketheatre',),
 ('likeconcerts',),
 ('likejazz',),
 ('likeclassical',),
 ('likeopera',),
 ('likerock',),
 ('likevegas',),
 ('likebroadway',),
 ('likemusicals',)]

In [182]:
get_column_names(table_name='sales')

[('salesid',),
 ('listid',),
 ('sellerid',),
 ('buyerid',),
 ('eventid',),
 ('dateid',),
 ('qtysold',),
 ('pricepaid',),
 ('commission',),
 ('saletime',)]

In [183]:
# Join users information to each sale
sq = """
select * from users
left join sales on sales.sellerid = users.userid
order by users.userid"""
run_query(sq)

(['userid',
  'username',
  'firstname',
  'lastname',
  'city',
  'state',
  'email',
  'phone',
  'likesports',
  'liketheatre',
  'likeconcerts',
  'likejazz',
  'likeclassical',
  'likeopera',
  'likerock',
  'likevegas',
  'likebroadway',
  'likemusicals',
  'salesid',
  'listid',
  'sellerid',
  'buyerid',
  'eventid',
  'dateid',
  'qtysold',
  'pricepaid',
  'commission',
  'saletime'],
 [(1,
   'JSG99FHE',
   'Rafael',
   'Taylor',
   'Kent',
   'WA',
   'Etiam.laoreet.libero@sodalesMaurisblandit.edu',
   '(664) 602-4412',
   True,
   True,
   None,
   False,
   True,
   None,
   None,
   True,
   False,
   True,
   94895,
   108334,
   1,
   23968,
   8344,
   2112,
   2,
   Decimal('494.00'),
   Decimal('74.10'),
   datetime.datetime(2008, 10, 13, 2, 19, 5)),
  (1,
   'JSG99FHE',
   'Rafael',
   'Taylor',
   'Kent',
   'WA',
   'Etiam.laoreet.libero@sodalesMaurisblandit.edu',
   '(664) 602-4412',
   True,
   True,
   None,
   False,
   True,
   None,
   None,
   True,
   Fal

In [184]:
# For each event attach the venue information
sq = """select * from event left join venue on venue.venueid = event.venueid order by event.venueid"""
run_query(sq)

(['eventid',
  'venueid',
  'catid',
  'dateid',
  'eventname',
  'starttime',
  'venueid',
  'venuename',
  'venuecity',
  'venuestate',
  'venueseats'],
 [(7919,
   1,
   9,
   2049,
   'Wyclef Jean',
   datetime.datetime(2008, 8, 11, 19, 0),
   1,
   'Toyota Park',
   'Bridgeview',
   'IL',
   0),
  (5356,
   1,
   9,
   2102,
   'B.B. King',
   datetime.datetime(2008, 10, 3, 15, 0),
   1,
   'Toyota Park',
   'Bridgeview',
   'IL',
   0),
  (4010,
   1,
   9,
   2031,
   'Allman Brothers Band',
   datetime.datetime(2008, 7, 24, 20, 0),
   1,
   'Toyota Park',
   'Bridgeview',
   'IL',
   0),
  (5117,
   1,
   9,
   1959,
   'Cute Is What We Aim For',
   datetime.datetime(2008, 5, 1, 19, 0),
   1,
   'Toyota Park',
   'Bridgeview',
   'IL',
   0),
  (7448,
   1,
   9,
   2046,
   'Lenny Kravitz',
   datetime.datetime(2008, 8, 8, 19, 30),
   1,
   'Toyota Park',
   'Bridgeview',
   'IL',
   0),
  (6625,
   1,
   9,
   1865,
   'Carrie Underwood',
   datetime.datetime(2008, 2, 8, 19, 

## Subqueries - 5 points

To wrap up let's do several subqueries. Please do the following:

* Get all purchases made by users of live in Arizona
* Get event information for all events that took place in a venue where the venue name ends with 'Stadium'. 
* Get event information for all events where the total ticket sales were greater than $50,000.  

In [185]:
# Get all purchases from users who live in Arizona
sq = """
select * from sales where sales.buyerid in (select userid from users where state in ('AZ'))
"""
run_query(sq)

(['salesid',
  'listid',
  'sellerid',
  'buyerid',
  'eventid',
  'dateid',
  'qtysold',
  'pricepaid',
  'commission',
  'saletime'],
 [(43,
   47,
   49346,
   33489,
   8577,
   2141,
   2,
   Decimal('378.00'),
   Decimal('56.70'),
   datetime.datetime(2008, 11, 11, 9, 51, 6)),
  (79,
   101,
   37592,
   7079,
   3340,
   1878,
   1,
   Decimal('36.00'),
   Decimal('5.40'),
   datetime.datetime(2008, 2, 21, 10, 32, 10)),
  (81,
   103,
   26314,
   7079,
   15,
   2033,
   1,
   Decimal('181.00'),
   Decimal('27.15'),
   datetime.datetime(2008, 7, 26, 11, 4, 13)),
  (83,
   106,
   12538,
   7079,
   250,
   1884,
   1,
   Decimal('109.00'),
   Decimal('16.35'),
   datetime.datetime(2008, 2, 27, 11, 58, 35)),
  (154,
   162,
   27703,
   46451,
   2906,
   1907,
   3,
   Decimal('426.00'),
   Decimal('63.90'),
   datetime.datetime(2008, 3, 22, 5, 21, 40)),
  (173,
   176,
   3941,
   10086,
   3097,
   2155,
   2,
   Decimal('40.00'),
   Decimal('6.00'),
   datetime.datetime(2008

In [186]:
# Get event information for all events that took place in a venue where the name ended in 'Stadium'
sq = """select * from event where event.venueid in (select venueid from venue where venuename like '%Stadium')"""
run_query(sq)

(['eventid', 'venueid', 'catid', 'dateid', 'eventname', 'starttime'],
 [(3803,
   2,
   9,
   2181,
   'Dropkick Murphys',
   datetime.datetime(2008, 12, 21, 14, 0)),
  (3816, 11, 9, 2139, 'Keb Mo', datetime.datetime(2008, 11, 9, 19, 0)),
  (3821,
   79,
   9,
   1885,
   'Charlie Daniels Band',
   datetime.datetime(2008, 2, 28, 19, 30)),
  (3824, 98, 9, 1885, 'Govt Mule', datetime.datetime(2008, 2, 28, 14, 0)),
  (3835, 74, 9, 2073, 'LeAnn Rimes', datetime.datetime(2008, 9, 4, 14, 30)),
  (3837, 67, 9, 1850, 'Randy Travis', datetime.datetime(2008, 1, 24, 15, 0)),
  (3840, 14, 9, 1876, 'Gwen Stefani', datetime.datetime(2008, 2, 19, 19, 30)),
  (3843, 75, 9, 1986, 'Blake Shelton', datetime.datetime(2008, 6, 9, 14, 30)),
  (3854, 6, 9, 1982, 'Gwen Stefani', datetime.datetime(2008, 6, 5, 15, 0)),
  (3862, 116, 9, 1855, 'Teddy Geiger', datetime.datetime(2008, 1, 29, 19, 30)),
  (3864, 75, 9, 1992, 'Celine Dion', datetime.datetime(2008, 6, 15, 15, 0)),
  (3866, 72, 9, 1950, 'Cold War Kids',

In [187]:
# Get event information where the total sales for that event were greater than $50000
sq = """select * from event where eventid in (select eventid from sales group by eventid having sum(pricepaid) > 50000)"""
run_query(sq)

(['eventid', 'venueid', 'catid', 'dateid', 'eventname', 'starttime'],
 [(289,
   300,
   8,
   2100,
   'Adriana Lecouvreur',
   datetime.datetime(2008, 10, 1, 19, 30)),
  (1602,
   257,
   6,
   2128,
   'Phantom of the Opera',
   datetime.datetime(2008, 10, 29, 19, 30)),
  (7895, 47, 9, 1944, 'Janet Jackson', datetime.datetime(2008, 4, 28, 15, 0))])

In [188]:
get_table_names()

[('users',),
 ('venue',),
 ('category',),
 ('date',),
 ('event',),
 ('listing',),
 ('sales',)]

In [189]:
get_column_names(table_name= 'date')

[('dateid',),
 ('caldate',),
 ('day',),
 ('week',),
 ('month',),
 ('qtr',),
 ('year',),
 ('holiday',)]

In [190]:
sq = """ SELECT * FROM date
          LIMIT 5;"""
run_query(sq)

(['dateid', 'caldate', 'day', 'week', 'month', 'qtr', 'year', 'holiday'],
 [(1827, datetime.date(2008, 1, 1), 'WE ', 1, 'JAN  ', '1    ', 2008, True),
  (1828, datetime.date(2008, 1, 2), 'TH ', 1, 'JAN  ', '1    ', 2008, False),
  (1829, datetime.date(2008, 1, 3), 'FR ', 1, 'JAN  ', '1    ', 2008, False),
  (1830, datetime.date(2008, 1, 4), 'SA ', 2, 'JAN  ', '1    ', 2008, False),
  (1831, datetime.date(2008, 1, 5), 'SU ', 2, 'JAN  ', '1    ', 2008, False)])